# Ch `09`: Concept `03`

## Convolution Neural Network

Load data from CIFAR-10.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cifar_tools
import tensorflow as tf

learning_rate = 0.001

names, data, labels = \
    cifar_tools.read_data('./cifar-10-batches-py')



names ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(50000, 3072) (50000,)


Define the placeholders and variables for the CNN model:

In [2]:
x = tf.placeholder(tf.float32, [None, 24 * 24])
y = tf.placeholder(tf.float32, [None, len(names)])
W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))
W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))
W3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))
W_out = tf.Variable(tf.random_normal([1024, len(names)]))
b_out = tf.Variable(tf.random_normal([len(names)]))

Define helper functions for the convolution and maxpool layers:

In [3]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out


def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

The CNN model is defined all within the following method:

In [4]:
def model():
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])

    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)

    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)

    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

Here's the cost function to train the classifier.

In [5]:
model_op = model()

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y)
)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Let's train the classifier on our data:

In [6]:
%%time

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size', batch_size)
    for j in range(0, 1000):
        avg_accuracy_val = 0.
        batch_count = 0.
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size, :]
            batch_onehot_vals = onehot_vals[i:i+batch_size, :]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y: batch_onehot_vals})
            avg_accuracy_val += accuracy_val
            batch_count += 1.
        avg_accuracy_val /= batch_count
        print('Epoch {}. Avg accuracy {}'.format(j, avg_accuracy_val))


batch size 250
Epoch 0. Avg accuracy 0.18456000205129386
Epoch 1. Avg accuracy 0.24773999854922293
Epoch 2. Avg accuracy 0.27087999761104586
Epoch 3. Avg accuracy 0.28337999783456325
Epoch 4. Avg accuracy 0.2953599992394447
Epoch 5. Avg accuracy 0.30531999841332436
Epoch 6. Avg accuracy 0.31325999870896337
Epoch 7. Avg accuracy 0.32047999896109103
Epoch 8. Avg accuracy 0.3274199993163347
Epoch 9. Avg accuracy 0.33337999783456324
Epoch 10. Avg accuracy 0.33811999917030333
Epoch 11. Avg accuracy 0.3442199990153313
Epoch 12. Avg accuracy 0.3495800018310547
Epoch 13. Avg accuracy 0.35466000124812125
Epoch 14. Avg accuracy 0.35827999949455264
Epoch 15. Avg accuracy 0.3606799976527691
Epoch 16. Avg accuracy 0.3638799959421158
Epoch 17. Avg accuracy 0.37039999827742576
Epoch 18. Avg accuracy 0.3693599990010262
Epoch 19. Avg accuracy 0.3804399982094765
Epoch 20. Avg accuracy 0.3807999996840954
Epoch 21. Avg accuracy 0.37851999670267106
Epoch 22. Avg accuracy 0.3806799973547459
Epoch 23. Avg ac